# 7월 28일

- VPC endpoint(일반적으로 엔드 포인트는 서비스 접점, 서비스를 연결할 수 있는 도메인을 말한다.)

# 17. 부록
## VPC Endpoint
- 프라이빗 네트워크 서비스를 도와주는 AWS 서비스의 기능
- VPC에 구멍을 뚫어 놓는 느낌
- **NAT, IGW를 거치지 않고도 해당 서비스간에 Private 통신을 지원하는 것이 VPC Endpoint이다.**
- 지원하는 서비스: 몇몇 개의 서비스들이 지원. 모든 서비스에서 지원하지 않는다. (점점 추가중)

### VPC Endpoint 유형
- 인터페이스
    - 네트워크 카드 하나가 추가되어 같은 세그먼트에 연결되어 있는 것처럼 지원
    - ENI를 사용하여 EC2가 서비스도 같은 서브넷 안에 있다고 판단하도록 구성한다. 서브넷 안에 ENI, ENI와 API 게이트웨이가 연결
    - S3와 대부분의 서비스

- 게이트웨이
    - 해당 서비스와 EC2 인스턴스 간에 전용 게이트웨이가 존재하는 것처럼 지원
    - S3, DynamoDB만 지원

### 프라이빗 노드 접근
- Bastion을 통한 프라이빗 노드 접근
- public subnet에 Bastion Host를 두고 외부에서 접근할 때 Bastion Host를 통해 private 서브넷에 접근하도록 한다.

- ssh 연결을 할 때 두 단계에 거쳐 이루어지도록 설정

### 실습13 - VPC Endpoint 구성(S3)
- 최종적으로 프라이빗 노드에서 게이트웨이를 거치지 않고 S3에 연결할 수 있도록 구성

- 퍼블릭 노드(Bastion)로 ssh 연결
    - S3에 미리 업로드 한 ssh 키 파일 다운로드(CLI 접근) --> Bastion에게 프라이빗 키 제공
    - 퍼블릭 노드에서 프라이빗 노드로 ssh 연결

- 프라이빗 노드(PriNode)에서 S3 접근(NAT GW 없는 환경이라 가정)
    - EC2(프라이빗노드)에서 S3로 접근
    - $ aws configure
    - $ aws s3 ls --> NAT가 없기 때문에 확인할 수 없다. 응답 없음

- VPC 엔드포인트 생성 : VPC > 엔드포인트 >> 엔드포인트 생성
    - AWS 서비스 : S3 
    - 유형 : Gateway
    - **라우팅 테이블 : 기존 private 라우팅 테이블 연결 후 확인**
    - pl(프라이빗 링크) 라우팅 목록이 생성된 것을 확인할 수 있다. 

- VPC 엔드포인트를 이용한 프라이빗 접근
    - EC2(프라이빗노드)에서 S3로 접근
    - $ aws s3 ls --> O. VPC endpoint로 인해 NAT 없이 통신이 되는 것을 확인할 수 있다.

- 마무리
    - VPC 엔드포인트 삭제, S3 삭제, EC2들 종료

## AWS 컨테이너 서비스
### 컨테이너
- 가상화 방식의 무겁고 느린 애플리케이션 서비스를 해결하기 위한 프로세스 격리 방안
    - 가상 머신 기반에서 서비스를 하는 것은, VM을 새로 생성하여 OS를 각각 설치하여 올려야 하는데 이 방식은 리소스 낭비가 심하고 불필요한 과정이다.
    - 개별적으로 OS를 설치하고 부팅하는 데 시간이 오래 걸리기 때문에 컨테이너를 사용한다.

- 애플리케이션 코드 + 런타임 + 라이브러리 + 시스템 도구 등을 하나의 인스턴스로 패키징
    - 애플리케이션을 컨테이너 형태로 hooking하여 사용한다.
    - 어떤 환경에서도 동일하게 동작할 수 있다.
    - 애플리케이션 자체도 가벼워지고 러닝 속도도 빨라져 기존의 방법보다 효율적이다.
    - 컨테이너 오케스트레이션: 컨테이너를 자동으로 시작, 중지 및 관리
        - ECS, EKS

- 온프레미스 환경에서 주로 사용하는 컨테이너 방식을 AWS에서도 지원한다!
    - EKS, ECS 서비스
        - 노드 장애 시 대처, 크기 조정 등을 자동으로 관리해주는 서비스
    - ECR 
        - 컨테이너 이미지 저장

- 컨테이너는 서버단을 아예 신경쓰지 않게 해주는 서비스이다.
    - EC2 기반으로 동작한다.

### 컨테이너 오케스트레이션
- 컨테이너 배포, 크기 조정, 모니터링, 장애 등 관리를 자동화



### ECS(Elastic Container Service)
- 컨테이너화 된 애플리케이션의 쉬운 배포, 관리, 크기 조정에 도움을 주는 관리형 컨테이너 오케스트레이션 서비스

-  ECS 시작 유형
    - ECS : EC2 기반의 컨테이너 조정, EC2 비용 지불
    - Fargate : 서버리스 컨테이너 서비스
        - 인스턴스 구성 및 관리 불필요


### ECR(Elastic Container Registry)
- 컨테이너 이미지 저장 서비스
- 온프레미스에서는 깃헙에 저장하는데, AWS에서는 ECR에 저장할 수 있다.

# 14. 서비스 모니터링하기
- 모니터링이란?
- 주요 모니터링 항목
- CloudWatch
- 리소스 모니터링하기
- CloudTail

## 모니터링이란?
### 모니터링
- 장애에 대한 미연 방지 및 사고에 대한 신속 대응을 위한 서비스 상태 파악
- 모니터링을 통해 긴급한 경우 알림을 받을 수 있도록 하기 위함
- 지속적인 정보 수집

### 주요 모니터링 항목
- 생사 모니터링
- CPU 사용률
- 메모리 사용률
- 디스크 용량
- 네트워크 트래픽

### CloudWatch
- cloudwatch의 주요 기능
    - 수집: 지속해서 정보를 수집한다. 리소스와 관계된 로그를 실시간으로 수집하고 기록한다.
    - 모니터링: 집중 관리 기능을 제공한다. 수집한 정보를 보기 쉽게 그래프로 표현하거나, 이러한 그래프를 모아 한 장소에서 열람할 수 있다.
    - 액션: 주로 경보 기능을 제공한다. 경보는 SNS, 메일, API 호출 등 다양한 형태로 이용자가 즉시 확인하기 쉽게 제공할 수 있다.

- 지표: 가시적으로 보여주는 기능으로, 그래프 등의 형태가 있다.
- 경보(알람): 임계치가 있어 이를 넘어서거나 이하로 내려가면 알려주는 기능이다. 
    - 예: 임계치 3, 연속해서 3번이 되면 알람으로 설정
- 로그
- 이벤트

### 리소스 모니터링하기
- 대시보드에서 원하는 옵션들을 선택하여 관심있는 서비스의 지표들을 한 곳에서 볼 수 있다. --> 위젯
- 경보의 SNS 주제 선택에서 알람을 수신할 SNS 설정

### Amazone CloudTrail
- AWS 모든 이벤트 로그 수집 서비스 (+콘솔 로그인 이벤트)
- 추적 생성: S3에 기록
- 기본적으로 90일 저장, 추가로 저장하고 싶으면 추적 생성을 이용하여 데이터를 90일 이상 저장할 수 있다. --> 추가 비용 부과.
- WHO
- WHEN
- WHAT
- JSON 형태로 기록. 누가, 언제, 무엇을, 어떤 리소스에 대해 작업했는지 등이 기록된다.
- 일종의 감사 자료로 사용 가능하다.


### 실습 - CloudTrail을 통한 감사
- 이벤트 로그 확인 및 추적 새성
    - CloudTrail --> 이벤트 기록에서 확인
    - 추적 생성으로 로그 저장. 별도로 지우지 않는 이상 S3에 지속적으로 기록
        - 로그 파일 SSE-KMS암호화 비활성화, 나머지 기본값
- 마무리: 추적 삭제, S3 버킷 및 객체 삭제



### 실습 18 - CloudWatch 지표 확인 및 알람 설정
- CloudWatch : 지표, 경보
    - 지표 확인: CloudWatch --> 지표 --> 모든 지표 --> EC2 지표 내용 확인 가능
    - 경보 설정
    - CloudWatch --> 경보 --> 경보 생성
        - EC2 지표가 이상치를 기록하면 알림(Simple Notification Service)을 통해 확인
    
    - EC2 부하 테스트
        - sudo yum unstall stress
        - sudo tress --timeout 600 --cpu1
            - 의도적으로 CPU에 부하를 가하여 지표 확인
    
    - 마무리
        - 알람 삭제, SNS topic 삭제

### 요약 키워드
- 모니터링이란?
- 주요 모니터링 항목
- CloudWatch를 통한 AWS 리소스 모니터링
- CloudTrail을 이용한 로그 수집


# 15. 요금 확인하기
- 요금 관리 방식
- Billing and Cost Management
- 예산 작성하기
- 일일 요금 확인하기
- 청구서 확인하기

- 불필요한 비용, 낭비하는 비용, 절약할 수 있는 비용 등이 없는지 확인해야 한다.

## 요금 관리 방식
- 계획 --> 실행 --> 평가 --> 개선을 반복하며 비용을 줄일 수 있는 작업을 진행


## Billing and Cost Management
- 예상 비용: AWS 요금 계산기
- 예산: 예산
- 월 요금: 비용 탐색기, 경보
- 판단: 펑구, CloudWatch
- 개선

- 요금 계산기를 통해 어느 정도의 예측이 가능하다.
    - 어떤 서비스를 사용할건지 선택 및 설정
    - 리소스별 예상 요금 비율 확인 가능
    

## 예산 작성하기
- 예상 요금을 바탕으로 예산 설정
- Billing Console --> 예산 --> 예산 설정
- 기본 예산, 예산 임계치 설정 등
- 특정 임계 값을 넘어가면 알림 설정 기능

- 일일 요금 확인
    - Cost Explorer 뷰 선택 화면
    - 서비스별 일간 이용량 뷰 확인
    - 가시적으로 확인할 수 있다.

- 청구서 확인하기

- EstimatedCharges를 선택하여 비용에 대한 알람 설정도 가능하다.

- 개인적으로 AWS를 사용하게 된다면, 사용하는 것이 모두 돈으로 나가기 때문에 실습이나 테스트 하는 경우 마무리로 삭제해주는 것을 잊지 말기



# 10. 커스텀 도메인과 DNS 준비하기
- 도메인 이름이란?
- Route 53의 기능 알아보기
- Certificate Manager
- 도메인 이름 취득하기
- 퍼블릭 DNS에 리소스 정보 추가하기
- 프라이빗 DNS 준비하기
- SSL 서버 인증서 발행하기



## 도메인 이름이란?
- 사람이 이해하기 쉽도록 만든 주소이다.
- DNS를 통해 IP 주소로 변환되어 사용자가 IP주소를 모르더라도 통신할 수 있도록 도와준다.

- 장애 발생을 대비하여 중복 네임 서버를 설치하는 것도 고객이 해야 할 일이었는데, AWS가 이 부분을 도와준다. 이 기능을 하는 것이 Route53 서비스이다.

## Route 53
- AWS에서 도메인 이름에 해당하는 IP 주소를 resolving해주는 서비스
- 가용성, 확장성이 뛰어난 DNS 서비스

## Certificate Manager
- CA 기반 인증서를 받을 수 있다.
- SSL 서버 인증서 시스템

## 도메인 이름 취득하기
- 도메인 등록 시작
- 도메인 검색
- 연락처 세부 정보
- 일메일 검증 단계
- 확인과 구매
- 등록과 도메인 확인
- 호스팅 영역

## 퍼블릭 DNS에 리소스 정보 추가하기
- 호스트 영역 세부 정보
- 라우팅 정책 선택 - 어떤 라우팅 정책을 사용할건지 선택
- 레코드 정의
- bastion 퍼블릭 IP 주소
- 단순 레코드 정의
- 레코드 설정
- 단순 레코드 정의(로드 밸런서)
- 레코드 설정 내용을 route53에 반영

### Route 53 라우팅 정책
- 단순 라우팅: www.example.com에 대해서 IP 하나 혹은 두 개로 구성

- 가중치 기반 라우팅: 주소에 가중치를 두고 전체 접근 중 가중치에 따라 분산하여 라우팅

- 지연시간 라우팅: 클라이언트 기준으로 지연시간이 짧은 곳으로 라우팅

- 지리 위치 라우팅: 클라이언트 위치 기준으로 가까운 곳으로 라우팅

- 다중 응답 라우팅: 요청에 대해 여러 개의 주소를 주는 라우팅 방식

- **장애 조치 라우팅**: 
    - IP 주소 두 개를 등록하면 하나는 Primary, 하나는 Secondary IP로 지정하여 Primary IP에 장애가 생기면 Secondary IP를 통해 접근이 가능하도록 라우팅하는 방식이다. 
    - 평상시에 Secondary는 Stanby 상태로 대기한다. 애플리케이션의 고가용성을 보장할 수 있다.
    - 리전 장애에 대비하는 애플리케이션 고가용성을 위해 서로 다른 리전에 구성한다.
    - 비용이 증가한다는 단점이 존재한다.
    - 무조건 리전을 분리해야 하는 것은 아니다.

- 클라이언트 입장에서는 연결이 잘 되기만 하면 문제 없기 때문에 이러한 라우팅 정책들을 사용할 수 있다.

- 프라이빗 DNS
    - 도메인 이름을 IP 주소로 변환하는 과정에서 보안과 프라이버시를 강화

# 18. 부록B – AWS 보안 개요
- 클라우드 보안 개요
- AWS 로그 수집
- AWS 주요 보안 서비스



## 클라우드 보안
### 보안의 3요소
- 기밀성(Confidentiality)
    - 인가된 사용자에게만 허가
    - 비밀스럽게 유지되며, 특정 인물만 볼 수 있는 기밀정보

- 무결성
    - 부적절한 정보 변경이나 삭제로부터 보호

- 가용성
    - 신뢰할 수 있는 정보의 접근과 사용

- 유연성 --> IaC: CloudFormation, 자동화 도구. 클라우드와 다르게 온프레미스 환경이라면 일정 주기마다 인프라 유지관리를 지속적으로 해야 하는 부담이 있음. 그러나, 클라우드 환경에서는 생성과 삭제가 쉬워 코드만 가지고 실행하면 생성 유지할 필요 없이 똑같은 환경으로 생성이 가능하다.
    - 1회 용품처럼 사용할 수 있는 것이 클라우드의 강점

- DoS 공격은 서비스에 대해 정상적인 요청을 폭주시켜 리소스를 고갈 시키는 공격인데, 온프레미스 환경에서는 전산실 데이터센터의 한정적인 리소스를 가지고 유연하게 확장하는 게 클라우드보다 제한적이다.
    - 앞단에서 필터링하기 때문에 실질적인 위험은 낮다.


- CIA 보안: 인가된 사용자만이 손상되지 않은 상태에서 필요할 때 사용할 수 있어야 한다.

### AWS 보안 혜택
- 안전한 데이터 유지
- 규정 준수 요구 준수
- 비용 절감
- 빠르고 간편한 확장

### 공동 책임 모델
- 고객: 방화벽 설정, IAM 설정을 책임진다.
- AWS: 하드웨어, 소프트웨어 부분을 책임진다.

### AWS 보안을 고려한 설계 접근 방식
- 요구사항 이해
- 요구사항에 맞춰 안전한 환경 구성
    - 예: S3 객체 데이터 암호화, IAM 권한 설정, 보안그룹 설정 등
- 템플릿 사용 강제 적용
    - 자동화된 코드로 동일한 권한, 설정으로 만들 수 있도록 템플릿 소스 코드를 생성하여 적용한다.
- **검증 활동 수행**을 통해 개선할 부분을 찾는다.

### 보안 향상 고려 사항
1. 계정 정보 정확히
2. MFA 사용
3. 하드 코딩 금지
4. 보안 그룹 제한
5. 전략적인 데이터 정책
6. 로그 수집해두기
7. Role 기반으로 IAM 사용
8. GuardDuty(위협 탐지 도구)이용하여 액션 취하기
9. 하나의 유저에 대해서 두 개의 키(액세스 키, 시크릿 액세스 키) 생성이 가능하여 주기적으로 교체
10. 전체적인 인프라 구성 환경에 있어서 기준을 가지고 만들기

# 정리
1. AWS 네트워크 환경에서 프라이빗하게 통신할 수 있도록 해주는 기능 중 하나인 VPC Endpoint
    - 인터페이스 유형: 네트워크 카드를 하나 더 꽂은 것처럼 지원하는 기능
    - 게이트웨이 유형: NAT 없이도 VPC 밖에 있는 서비스에 연결할 수 있도록 해주는 기능

2. CloudTrail 서비스
    - 감사 자료로 활용할 수 있도록 로그를 저장하는 서비스
    - 추적 생성을 통해 S3에 저장해둘 수 있다.

3. CloudWatch 서비스
    - 여러 지표를 확인할 수 있다. (예: CPU 사용률)
    - 임계치를 설정하여 이를 넘어서거나 내려가면 알림(SNS)를 받아 조치할 수 있다.
    - 이 외에도 다양한 모니터링 도구들이 존재하여 개선점을 찾을 수 있다.

4. Route 53
    - 호스트 영역에서 도메인 등록을 하고, IP를 등록한다. 
    - Name Server인 Route53에 요청하면 DNS 네임서버로의 역할을 한다.
    - 하나의 이름 주소에 다양한 IP 주소 등록이 가능한데, 라우팅 정책에 따라 우선순위를 지정하여 제공한다.

5. 클라우드 보안
    - 온프레미스 환경과 클라우드 환경에서의 차이
    - 보안의 3요소
    - 보안 향상 사항
